# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121325e+02     2.091646e+00
 * time: 0.1351470947265625
     1     1.100370e+01     1.089591e+00
 * time: 1.0969820022583008
     2    -1.179533e+01     1.281672e+00
 * time: 1.2475922107696533
     3    -3.411238e+01     9.275418e-01
 * time: 1.417978048324585
     4    -4.771241e+01     5.444966e-01
 * time: 1.5770671367645264
     5    -5.699195e+01     1.833667e-01
 * time: 1.726938009262085
     6    -5.983043e+01     1.192086e-01
 * time: 1.854283094406128
     7    -6.084154e+01     4.763431e-02
 * time: 1.9799151420593262
     8    -6.121167e+01     6.872577e-02
 * time: 2.0966901779174805
     9    -6.152699e+01     4.164062e-02
 * time: 2.2195792198181152
    10    -6.173852e+01     2.961087e-02
 * time: 2.336005210876465
    11    -6.191605e+01     2.559027e-02
 * time: 2.4624969959259033
    12    -6.199473e+01     2.595821e-02
 * time: 2.5904340744018555
    13    -6.207806e+01     1.822825e-02
 * time: 2.70576214790344

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557710
    AtomicNonlocal      14.8522670
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485381 
    Xc                  -19.3336824

    total               -62.261666461186
